<a href="https://colab.research.google.com/github/ankit-rathi/Data-Science-with-Python/blob/master/Tradevesting/myProspects_Acc_Dis_Signals.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# install libraries
!pip install yfinance
!pip install pandas-ta
#!pip install talib-binary

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
# fetch nifty200 scrips

import pandas as pd
import numpy as np
from google.colab import drive
drive.mount('/content/drive')

mypf = pd.read_csv('/content/drive/My Drive/data/stocks/myProspectsScrips.csv')

#mypf = mypf[mypf['Nifty100'] == 1]

stock_n100 = mypf['Symbol'].unique()

exclude = ['CADILAHC','MMTC']

stock_n100 = mypf[~mypf['Symbol'].isin(exclude) ]['Symbol'].unique()

stock_ids = stock_n100

#mypf = mypf[mypf.Forecast.notnull()]
#stock_ids = mypf['Symbol'].unique()

stock_ids.sort()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
# star stocks & my stocks
star_stocks = list(mypf[mypf['StarStock'] == 1]['Stock'].values)
my_stocks = list(mypf[mypf['InPortfolio'] == 1]['Symbol'].values)

In [4]:
# import libraries

%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime
from tabulate import tabulate
import warnings
warnings.filterwarnings('ignore')
import pandas_datareader.data as web
import yfinance as yfin
#import talib
yfin.pdr_override()
import pandas_ta as ta

In [5]:
# set start and end date
import datetime
start_date = (datetime.date.today() + datetime.timedelta(days=-365*5)).strftime('%Y-%m-%d')
end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')

In [6]:
# function to compute stock attributes

def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    # import the closing price data of the stock for the aforementioned period of time in Pandas dataframe
    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))
    stock_df = yfin.Ticker(stock_symbol).history(period='5y')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    #stock_df = pd.DataFrame(stock_df) # convert Series object to dataframe
    #stock_df.columns = {'Close Price'} # assign new colun name
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    # column names for long and short moving average columns
    short_window_col = str(short_window) + '_' + moving_avg
    mid_window_col = str(mid_window) + '_' + moving_avg
    long_window_col = str(long_window) + '_' + moving_avg

    #print(stock_df)

    # Create short exponential moving average column
    stock_df[short_window_col] = round(stock_df['Close'].rolling(window = short_window, min_periods = 1).mean(),0)

    stock_df[mid_window_col] = round(stock_df['Close'].rolling(window = mid_window, min_periods = 1).mean(),0)

    #stock_df['RSI'] = round(talib.RSI(stock_df['Close Price'], timeperiod=14),0)

    # Create a long exponential moving average column
    stock_df[long_window_col] = round(stock_df['Close'].rolling(window = long_window, min_periods = 1).mean(),0)
    stock_df['Stock'] = stock_symbol
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df['200_SMA_Lag10'] = stock_df['200_SMA'].shift(10)
    stock_df['Slope%_200'] = round((stock_df['200_SMA'] - stock_df['200_SMA_Lag10'])*100/stock_df['200_SMA'],2)
    stock_df['50_SMA_Lag5'] = stock_df['50_SMA'].shift(5)
    stock_df['Slope%_50'] = round((stock_df['50_SMA'] - stock_df['50_SMA_Lag5'])*100/stock_df['50_SMA'],2)

    #stock_df.ta.rsi(append=True)
    #stock_df.rename(columns={'RSI_14':'RSI'}, inplace=True)
    stock_df['Close'] = round(stock_df['Close'],0)
    #stock_df['RSI'] = round(stock_df['RSI'],0)
    stock_df['Max'] = round(max(stock_df['Close']),0)
    stock_df['Min'] = round(min(stock_df['Close']),0)
    stock_df['Discount%'] = round((stock_df['Max']-stock_df['Close'])*100/(stock_df['Max']-stock_df['Min']),2)
    stock_df['Upside%ATH'] = round((stock_df['Max']-stock_df['Close'])*100/(stock_df['Close']),0)
    stock_df.drop(['Open', 'Low', 'High', 'Volume','20_SMA','50_SMA','200_SMA','200_SMA_Lag10','50_SMA_Lag5'
                   ], axis=1, inplace=True)

    return stock_df

df_prec_dev = pd.DataFrame()

#tmp = stock_prec_dev('ULTRACEMCO.NS')
#df_perform = df_perform.append(tmp)
for stock_id in stock_ids:

    stock_id = stock_id.upper() + '.NS'
    tmp = stock_prec_dev(stock_id)
    tmp = tmp.reset_index()
    tmp = tmp.tail(1)
    tmp['Avg_Cost'] = mypf[(mypf['Symbol']+'.NS') == stock_id.upper()]['AvgCost'].values[0]
    tmp['Shares'] = mypf[(mypf['Symbol']+'.NS') == stock_id.upper()]['Shares'].values[0]
    tmp['Med_PE'] = mypf[(mypf['Symbol']+'.NS') == stock_id.upper()]['MedianPE'].values[0]
    #tmp['Curr_PE'] = current_pe
    #tmp['BizOutlook'] = mypf[(mypf['Symbol']+'.NS') == stock_id.upper()]['BizOutlook'].values[0]
    #tmp['BizDurability'] = mypf[(mypf['Symbol']+'.NS') == stock_id.upper()]['BizDurability'].values[0]
    tmp['Conviction'] = mypf[(mypf['Symbol']+'.NS') == stock_id.upper()]['Conviction'].values[0]
    tmp['LatestQtr'] = mypf[(mypf['Symbol']+'.NS') == stock_id.upper()]['LatestQtr'].values[0]
    tmp['StarStock'] = mypf[(mypf['Symbol']+'.NS') == stock_id.upper()]['StarStock'].values[0]
    tmp['Criteria'] = mypf[(mypf['Symbol']+'.NS') == stock_id.upper()]['Criteria'].values[0]
    tmp['Strategy'] = mypf[(mypf['Symbol']+'.NS') == stock_id.upper()]['Strategy'].values[0]
    df_prec_dev = df_prec_dev.append(tmp)

df_prec_dev

,Date,Close,Stock,Dev%_200,Slope%_200,Slope%_50,Max,Min,Discount%,Upside%ATH,Avg_Cost,Shares,Med_PE,Conviction,LatestQtr,StarStock,Criteria,Strategy
1233,2023-06-19 00:00:00+05:30,27813.0,3MINDIA.NS,20.27,1.17,2.01,29373.0,15332.0,11.11,6.0,NaN,NaN,NaN,M,1.0,1.0,V40N,NaN
1233,2023-06-19 00:00:00+05:30,362.0,5PAISA.NS,13.95,1.57,2.46,553.0,96.0,41.79,53.0,347.06,228.0,NaN,M,1.0,1.0,V40N,ATH
1160,2023-06-19 00:00:00+05:30,1372.0,AAVAS.NS,-25.04,-2.35,-1.84,3196.0,635.0,71.22,133.0,1390.60,23.0,NaN,M,1.0,1.0,2T3Y,2T3Y
1233,2023-06-19 00:00:00+05:30,23094.0,ABBOTINDIA.NS,12.13,0.80,0.17,23094.0,6634.0,0.00,0.0,NaN,NaN,NaN,M,1.0,1.0,V40,NaN
1233,2023-06-19 00:00:00+05:30,1825.0,ACC.NS,-14.22,-1.03,1.07,2748.0,863.0,48.97,51.0,NaN,NaN,NaN,L,0.0,0.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1233,2023-06-19 00:00:00+05:30,790.0,VOLTAS.NS,-5.82,-1.19,0.00,1316.0,419.0,58.64,67.0,821.00,NaN,43.4,L,1.0,0.0,NaN,NaN
1233,2023-06-19 00:00:00+05:30,1450.0,WHIRLPOOL.NS,-0.98,-1.23,1.09,2667.0,1254.0,86.13,84.0,NaN,NaN,72.2,M,1.0,0.0,V40,NaN
1233,2023-06-19 00:00:00+05:30,380.0,WIPRO.NS,-3.02,-0.26,0.78,713.0,160.0,60.22,88.0,388.07,195.0,20.9,L,1.0,0.0,2T3Y,2T3Y
1233,2023-06-19 00:00:00+05:30,16.0,YESBANK.NS,-4.41,0.00,0.00,383.0,11.0,98.66,2294.0,NaN,NaN,NaN,L,1.0,0.0,NaN,NaN


In [7]:
cols = ['Date', 'Close', 'Stock', 'Dev%_200', 'StarStock','Criteria']
df_prec_dev = df_prec_dev[cols]

In [8]:
df_acc_dis = pd.DataFrame()

for stock_id in stock_ids:
      stock_id = stock_id.upper() + '.NS'
      tmp = stock_prec_dev(stock_id)
      list1 = [stock_id, round(tmp[tmp['Dev%_200']<0]['Dev%_200'].mean(),0), round(tmp[tmp['Dev%_200']>0]['Dev%_200'].mean(),0)]
      tmp1 = pd.DataFrame(list1).T
      tmp1.columns = ['Stock', 'Acc', 'Dis']
      df_acc_dis = df_acc_dis.append(tmp1)

df_acc_dis

,Stock,Acc,Dis
0,3MINDIA.NS,-8.0,8.0
0,5PAISA.NS,-14.0,25.0
0,AAVAS.NS,-15.0,25.0
0,ABBOTINDIA.NS,-5.0,15.0
0,ACC.NS,-9.0,12.0
...,...,...,...
0,VOLTAS.NS,-12.0,14.0
0,WHIRLPOOL.NS,-9.0,13.0
0,WIPRO.NS,-12.0,22.0
0,YESBANK.NS,-30.0,12.0


In [9]:
df_recomm = pd.merge(df_prec_dev, df_acc_dis)
df_recomm

,Date,Close,Stock,Dev%_200,StarStock,Criteria,Acc,Dis
0,2023-06-19 00:00:00+05:30,27813.0,3MINDIA.NS,20.27,1.0,V40N,-8.0,8.0
1,2023-06-19 00:00:00+05:30,362.0,5PAISA.NS,13.95,1.0,V40N,-14.0,25.0
2,2023-06-19 00:00:00+05:30,1372.0,AAVAS.NS,-25.04,1.0,2T3Y,-15.0,25.0
3,2023-06-19 00:00:00+05:30,23094.0,ABBOTINDIA.NS,12.13,1.0,V40,-5.0,15.0
4,2023-06-19 00:00:00+05:30,1825.0,ACC.NS,-14.22,0.0,NaN,-9.0,12.0
...,...,...,...,...,...,...,...,...
177,2023-06-19 00:00:00+05:30,790.0,VOLTAS.NS,-5.82,0.0,NaN,-12.0,14.0
178,2023-06-19 00:00:00+05:30,1450.0,WHIRLPOOL.NS,-0.98,0.0,V40,-9.0,13.0
179,2023-06-19 00:00:00+05:30,380.0,WIPRO.NS,-3.02,0.0,2T3Y,-12.0,22.0
180,2023-06-19 00:00:00+05:30,16.0,YESBANK.NS,-4.41,0.0,NaN,-30.0,12.0


In [10]:
# Accumulate
df_recomm[df_recomm['Dev%_200'] < df_recomm['Acc']]

,Date,Close,Stock,Dev%_200,StarStock,Criteria,Acc,Dis
2,2023-06-19 00:00:00+05:30,1372.0,AAVAS.NS,-25.04,1.0,2T3Y,-15.0,25.0
4,2023-06-19 00:00:00+05:30,1825.0,ACC.NS,-14.22,0.0,NaN,-9.0,12.0
6,2023-06-19 00:00:00+05:30,960.0,ADANIGREEN.NS,-36.27,1.0,NaN,-22.0,58.0
8,2023-06-19 00:00:00+05:30,807.0,ADANITRANS.NS,-61.43,1.0,NaN,-28.0,39.0
62,2023-06-19 00:00:00+05:30,1024.0,GLAND.NS,-34.46,0.0,2T3Y,-25.0,19.0
82,2023-06-19 00:00:00+05:30,129.0,IEX.NS,-11.82,NaN,NaN,-11.0,28.0
89,2023-06-19 00:00:00+05:30,1294.0,INFY.NS,-8.36,1.0,V40,-8.0,15.0
135,2023-06-19 00:00:00+05:30,1423.0,PVRINOX.NS,-11.89,0.0,NaN,-8.0,3.0
137,2023-06-19 00:00:00+05:30,566.0,RAJESHEXPO.NS,-15.46,1.0,V40N,-12.0,12.0
175,2023-06-19 00:00:00+05:30,1841.0,VINATIORGA.NS,-6.46,0.0,V40N,-6.0,16.0


In [11]:
# Disburse
df_recomm[df_recomm['Dev%_200'] > df_recomm['Dis']]

,Date,Close,Stock,Dev%_200,StarStock,Criteria,Acc,Dis
0,2023-06-19 00:00:00+05:30,27813.0,3MINDIA.NS,20.27,1.0,V40N,-8.0,8.0
9,2023-06-19 00:00:00+05:30,2446.0,AKZOINDIA.NS,9.21,1.0,V40,-6.0,8.0
17,2023-06-19 00:00:00+05:30,772.0,AUBANK.NS,19.34,1.0,NaN,-11.0,15.0
18,2023-06-19 00:00:00+05:30,678.0,AUROPHARMA.NS,31.92,0.0,NaN,-16.0,18.0
21,2023-06-19 00:00:00+05:30,4597.0,BAJAJ-AUTO.NS,17.37,1.0,V40,-6.0,9.0
33,2023-06-19 00:00:00+05:30,375.0,BPCL.NS,11.87,0.0,NaN,-9.0,11.0
34,2023-06-19 00:00:00+05:30,5021.0,BRITANNIA.NS,19.15,1.0,NaN,-6.0,9.0
38,2023-06-19 00:00:00+05:30,7912.0,CERA.NS,32.36,1.0,V40N,-7.0,18.0
39,2023-06-19 00:00:00+05:30,1132.0,CHOLAFIN.NS,41.71,1.0,NaN,-15.0,20.0
48,2023-06-19 00:00:00+05:30,485.0,DLF.NS,23.19,0.0,NaN,-11.0,20.0
